In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import pandas as pd 
import numpy as np
import os

#import data 
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial
DATA_PATH = "/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial/raw_data/"
os.getcwd()

Mounted at /content/drive
/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial


'/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial'

In [2]:
from os import listdir
FILE_LIST = listdir(DATA_PATH)
print(FILE_LIST)

#test
#FILE_LIST[0].split(".csv")[0]

df_dic={} #create a dictionary with names of datasets as keys and datasets as values
for i, e in enumerate(FILE_LIST):
  df_dic[str(FILE_LIST[i].split(".csv")[0])]= pd.read_csv("raw_data/" + e)

#test
#df_dic['diaries_transactions_all'].columns

['con_prod_hh_over_trx_yr_mo_usd.csv', 'con_pur_hh_by_trx_yr_mo_usd.csv', 'demo_hh.csv', 'demo_ind.csv', 'diaries_goingson_hh.csv', 'diaries_wellbeing_ind.csv', 'edu_hh.csv', 'edu_ind.csv', 'housing_conditions_ownership.csv', 'inc_hh_by_trx_yr_mo_usd_incRR.csv', 'inc_ind_by_trx_yr_mo_usd_incRR.csv', 'pov_hh_by_trx_yr_mo_usd_incRR.csv', 'rem_hh_by_trx_yr_mo_usd.csv', 'pov_hh_by_trx_yr_mo_usd_excRR.csv', 'diaries_transactions_all.csv']


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
#start merging
#first we merge datasets with same keys together, then we merge the bigger (already merged) datasets together
from functools import reduce

dfs = [df_dic['diaries_transactions_all'], df_dic['demo_hh'] , df_dic['con_prod_hh_over_trx_yr_mo_usd'], df_dic['edu_hh']]

result_hhids = reduce(lambda df_left, df_right: pd.merge(df_left, df_right, 
                                              right_on = 'hh_ids', left_on = 'hh_ids', 
                                              how='outer'), 
                  dfs)

In [ ]:
print(result_hhids)

         hh_ids  unique_hhs first_trx_date_hh last_trx_date_hh  \
0       KELDL02         NaN         03sep2012        02oct2013   
1       KELDL02         NaN         03sep2012        02oct2013   
2       KELDL02         NaN         03sep2012        02oct2013   
3       KELDL02         NaN         03sep2012        02oct2013   
4       KELDL02         NaN         03sep2012        02oct2013   
...         ...         ...               ...              ...   
483944  KMOMT33         NaN         07oct2012        02nov2013   
483945  KMOMT33         NaN         07oct2012        02nov2013   
483946  KMOMT33         NaN         07oct2012        02nov2013   
483947  KMOMT33         NaN         07oct2012        02nov2013   
483948  KMOMT33         NaN         07oct2012        02nov2013   

        tot_hh_daysofobs  tot_hh_monthsofobs interview_designation   int_date  \
0                    394                  13  04=Diaries Interview  02oct2013   
1                    394                  13 

In [4]:
dfs = [df_dic['con_pur_hh_by_trx_yr_mo_usd'], df_dic['inc_hh_by_trx_yr_mo_usd_incRR'], df_dic['pov_hh_by_trx_yr_mo_usd_excRR'], df_dic['pov_hh_by_trx_yr_mo_usd_incRR'], df_dic['rem_hh_by_trx_yr_mo_usd']]

result_hhids_trxyrmo = reduce(lambda df_left, df_right: pd.merge(df_left, df_right, 
                                              right_on = ['hh_ids', 'trx_yr_mo'], left_on =['hh_ids', 'trx_yr_mo'], 
                                              how= 'outer'),
                  dfs)

In [ ]:
print(result_hhids_trxyrmo)

       hh_ids trx_yr_mo  con_pur_hh_byti_x  con_pur_hh_byti_ALC  \
0     KELDK01   2012_10           17.29412                  0.0   
1     KELDK01   2012_11           40.52941                  0.0   
2     KELDK01   2012_12           42.43530                  0.0   
3     KELDK01   2013_01           48.23529                  0.0   
4     KELDK01   2013_02          107.52940                  0.0   
...       ...       ...                ...                  ...   
3486  KNBOK36   2012_12                NaN                  NaN   
3487  KNBOM26   2013_11                NaN                  NaN   
3488  KVIHC09   2012_12                NaN                  NaN   
3489  KVIHC12   2012_10                NaN                  NaN   
3490  KNBOK06   2012_11                NaN                  NaN   

      con_pur_hh_byti_CLN  con_pur_hh_byti_CLTH  con_pur_hh_byti_COMM  \
0                0.000000              8.235294              0.470588   
1                0.000000              0.000000  

In [5]:
dfs = [df_dic['diaries_wellbeing_ind'], df_dic['diaries_goingson_hh']]

result_hhids_intdate = reduce(lambda df_left, df_right: pd.merge(df_left, df_right, 
                                              right_on = ['hh_ids', 'int_date'], left_on =['hh_ids', 'int_date'], 
                                              how= 'outer'),
                  dfs)


In [ ]:
print(result_hhids_intdate)

In [6]:
dfs = [df_dic['demo_ind'], df_dic['edu_ind']]

result_hhids_inds = reduce(lambda df_left, df_right: pd.merge(df_left, df_right, 
                                              right_on = ['hh_ids', 'm_ids'], left_on =['hh_ids', 'm_ids'], 
                                              how= 'outer'),
                  dfs)

In [ ]:
print(result_hhids_inds)

In [7]:
#final part of merging
official_1 = pd.merge(result_hhids, result_hhids_trxyrmo, right_on = ['hh_ids', 'trx_yr_mo'], left_on =['hh_ids', 'trx_yr_mo'], 
                                              how= 'outer')



In [8]:
official_2 = pd.merge(official_1, result_hhids_intdate, right_on = ['hh_ids', 'int_date'], left_on =['hh_ids', 'int_date'], 
                                              how= 'outer')

In [9]:
official= pd.merge(official_2, result_hhids_inds, right_on = ['hh_ids', 'm_ids'], left_on =['hh_ids', 'm_ids'], 
                                              how= 'outer')


In [10]:
official.head()

,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,...,dem_i_age_workage,dem_i_relhead,dem_i_hhead,dem_i_marstat,dem_i_tribe,dem_i_children_tot,dem_i_children_inhh,edu_i_enrolled,edu_i_attain,edu_i_attain_yrs
0,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,2013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,2013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,2013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,2013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,2013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
official[official['hh_ids'] == 'KELDK01']

,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,...,dem_i_age_workage,dem_i_relhead,dem_i_hhead,dem_i_marstat,dem_i_tribe,dem_i_children_tot,dem_i_children_inhh,edu_i_enrolled,edu_i_attain,edu_i_attain_yrs
368699,KELDK01,NaN,18sep2012,02oct2013,379.0,13.0,04=Diaries Interview,07aug2013,8.0,2013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
368700,KELDK01,NaN,18sep2012,02oct2013,379.0,13.0,04=Diaries Interview,07aug2013,8.0,2013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
368701,KELDK01,NaN,18sep2012,02oct2013,379.0,13.0,04=Diaries Interview,07aug2013,8.0,2013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
368702,KELDK01,NaN,18sep2012,02oct2013,379.0,13.0,04=Diaries Interview,07aug2013,8.0,2013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
368703,KELDK01,NaN,18sep2012,02oct2013,379.0,13.0,04=Diaries Interview,07aug2013,8.0,2013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370197,KELDK01,NaN,18sep2012,02oct2013,379.0,13.0,05=Module,04oct2013,10.0,2013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
734953,KELDK01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,Son or daughter,0.0,Never married/lived together,Kalenjin,NaN,NaN,1.0,Primary (some or complete),3.0
734954,KELDK01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,Household head,1.0,Married/living together,Kalenjin,2.0,0.0,0.0,Primary (some or complete),8.0
734955,KELDK01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,Son or daughter,0.0,Never married/lived together,Kalenjin,NaN,NaN,0.0,No education,0.0


final comment on merging --> DID NOT MANAGE TO MERGE inc_ind_by_trx_yr_mo_usd_incRR and housing_conditions --> remember to include limitations of this choice in our report!

In [12]:
#replacing NAs consistently along the whole dataset
official.replace('.', np.NaN)
official.replace(' ', np.NaN)

,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,...,dem_i_age_workage,dem_i_relhead,dem_i_hhead,dem_i_marstat,dem_i_tribe,dem_i_children_tot,dem_i_children_inhh,edu_i_enrolled,edu_i_attain,edu_i_attain_yrs
0,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,2013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,2013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,2013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,2013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KELDL02,NaN,03sep2012,02oct2013,394.0,13.0,04=Diaries Interview,02oct2013,10.0,2013.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736513,KZIMA04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,Son or daughter,0.0,Never married/lived together,Kikuyu,NaN,NaN,1.0,NaN,NaN
736514,KZIMA05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,Household head,1.0,Widowed,Kikuyu,0.0,1.0,NaN,NaN,NaN
736515,KZIMA08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,Husband or wife,0.0,Married/living together,Kamba,1.0,0.0,0.0,Secondary (some or complete),12.0
736516,KZIMA08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,Household head,1.0,Married/living together,Embu,0.0,1.0,0.0,Secondary (some or complete),12.0


In [13]:
#checking for empty columns 
empty_cols = [col for col in official.columns if official[col].isnull().all()]

print(empty_cols) #there are no empty columns

[]


Merged dataframe is ready to be split into train and test!

In [14]:
os.getcwd()

'/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial'

In [17]:
os.makedirs('clean_data', exist_ok=True)  
compression_opts = dict(method='zip',
                        archive_name='official.csv')
official.to_csv('clean_data/official.zip', index=False,
          compression=compression_opts) 